In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("hw7.ipynb")

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: [Mar 24, 11:59 pm](https://github.com/UBC-CS/cpsc330-2024W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [2]:
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

<!-- BEGIN QUESTION -->

<div class="alert alert-info">
    
## Submission instructions
<hr>
rubric={points}

**Please be aware that this homework assignment requires installation of several packages in your course environment. It's possible that you'll encounter installation challenges, which might be frustrating. However, remember that solving these issues is not wasting time but it is an essential skill for anyone aspiring to work in data science or machine learning.**

Follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2024W2/blob/main/docs/homework_instructions.md). 

**You may work in a group on this homework and submit your assignment as a group.** Below are some instructions on working as a group.  
- The maximum group size is 2. 
- Use group work as an opportunity to collaborate and learn new things from each other. 
- Be respectful to each other and make sure you understand all the concepts in the assignment well. 
- It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. 
- You can find the instructions on how to do group submission on Gradescope [here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members).


When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using Gradescope's drag and drop tool. Check out this [Gradescope Student Guide](https://lthub.ubc.ca/guides/gradescope-student-guide/) if you need help with Gradescope submission.
4. Make sure that the plots and output are rendered properly in your submitted file. 
5. If the .ipynb file is too big and doesn't render on Gradescope, also upload a pdf or html in addition to the .ipynb. If the pdf or html also fail to render on Gradescope, please create two files for your homework: hw6a.ipynb with Exercise 1 and hw6b.ipynb with Exercises 2 and 3 and submit these two files in your submission.  
</div>

_Points:_ 2

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 18, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [3]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [4]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

[=========-----------------------------------------] 18.3% 23.4/128.1MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



[==================================----------------] 69.8% 89.4/128.1MB downloaded

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [5]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words using `glove_wiki_vectors` embeddings.

<div class="alert alert-warning">

Solution_1.1
    
</div>

_Points:_ 2

In [7]:
words = ["king", "happy", "computer", "pizza"]

for word in words:
    print(f"Words similar to '{word}':")
    try:
        similar_words = glove_wiki_vectors.most_similar(word, topn=5)  # Get top 5 similar words
        for similar_word, similarity_score in similar_words:
            print(f"  - {similar_word} (similarity: {similarity_score:.2f})")
    except KeyError:
        print(f"  - '{word}' not in vocabulary")
    print()

Words similar to 'king':
  - prince (similarity: 0.77)
  - queen (similarity: 0.75)
  - son (similarity: 0.70)
  - brother (similarity: 0.70)
  - monarch (similarity: 0.70)

Words similar to 'happy':
  - 'm (similarity: 0.84)
  - feel (similarity: 0.81)
  - 're (similarity: 0.80)
  - i (similarity: 0.79)
  - 'll (similarity: 0.79)

Words similar to 'computer':
  - computers (similarity: 0.88)
  - software (similarity: 0.84)
  - technology (similarity: 0.76)
  - pc (similarity: 0.74)
  - hardware (similarity: 0.73)

Words similar to 'pizza':
  - sandwich (similarity: 0.75)
  - sandwiches (similarity: 0.72)
  - bread (similarity: 0.70)
  - pizzas (similarity: 0.70)
  - burger (similarity: 0.70)



In [8]:
...

Ellipsis

In [9]:
...

Ellipsis

In [10]:
...

Ellipsis

In [11]:
...

Ellipsis

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.2 Word similarity using pre-trained embeddings
rubric={points}

**Your tasks:**

1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [13]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

_Points:_ 2

In [15]:
for word1, word2 in word_pairs:
    try:
        similarity = glove_wiki_vectors.similarity(word1, word2)
        print(f"Similarity between '{word1}' and '{word2}': {similarity:.3f}")
    except KeyError as e:
        print(f"One or both words not in vocabulary: {word1}, {word2}")

Similarity between 'coast' and 'shore': 0.700
Similarity between 'clothes' and 'closet': 0.546
Similarity between 'old' and 'new': 0.643
Similarity between 'smart' and 'intelligent': 0.755
Similarity between 'dog' and 'cat': 0.880
Similarity between 'tree' and 'lawyer': 0.077


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Representation of all words in English
rubric={points}

**Your tasks:**

1. The vocabulary size of Wikipedia embeddings is quite large. The `test_words` list below contains a few new words (called neologisms) and biomedical domain-specific abbreviations. Write code to check whether `glove_wiki_vectors` have representation for these words or not. 
> If a given word `word` is in the vocabulary, `word in glove_wiki_vectors` will return True. 

In [17]:
test_words = [
    "covididiot",
    "fomo",
    "frenemies",
    "anthropause",
    "photobomb",
    "selfie",
    "pxg",  # Abbreviation for pseudoexfoliative glaucoma
    "pacg",  # Abbreviation for primary angle closure glaucoma
    "cct",  # Abbreviation for central corneal thickness
    "escc",  # Abbreviation for esophageal squamous cell carcinoma
]

<div class="alert alert-warning">

Solution_1_3
    
</div>

_Points:_ 2

In [18]:
for word in test_words:
    if word in glove_wiki_vectors:
        print(f"'{word}' is in the vocabulary.")
    else:
        print(f"'{word}' is NOT in the vocabulary.")

'covididiot' is NOT in the vocabulary.
'fomo' is NOT in the vocabulary.
'frenemies' is in the vocabulary.
'anthropause' is NOT in the vocabulary.
'photobomb' is NOT in the vocabulary.
'selfie' is NOT in the vocabulary.
'pxg' is NOT in the vocabulary.
'pacg' is NOT in the vocabulary.
'cct' is in the vocabulary.
'escc' is in the vocabulary.


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Stereotypes and biases in embeddings
rubric={points}

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore this. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

In [19]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

Examples of using analogy to explore biases and stereotypes.  

In [20]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


In [21]:
glove_wiki_vectors.similarity("aboriginal", "success")

0.1428324

In [22]:
glove_wiki_vectors.similarity("white", "success")

0.351824

<div class="alert alert-warning">

Solution_1_4
    
</div>

_Points:_ 4

In [27]:
print(glove_wiki_vectors.similarity("black", "crime"))   
print(glove_wiki_vectors.similarity("white", "crime"))   

0.43576175
0.38129753


"Black" has higher similarity to "crime" than "white," reflecting harmful racial biases in the training data.

In [43]:
analogy("he", "genius", "she")

he : genius :: she : ?


,Analogy word,Score
0,beauty,0.608836
1,lover,0.595915
2,love,0.573946
3,artistry,0.566379
4,obsession,0.564235
5,cinematic,0.560095
6,sensibility,0.555178
7,amazing,0.552442
8,heroine,0.550451
9,imagination,0.549747


The model associates "she" with "beauty" rather than intellectual traits ("genius", "scientist"). This reflects a gendered stereotype that prioritizes women’s appearance over intelligence.

In [49]:
analogy("gay", "flamboyant", "straight")

gay : flamboyant :: straight : ?


,Analogy word,Score
0,brilliant,0.603264
1,erratic,0.579939
2,dazzling,0.574534
3,stunning,0.571415
4,impressive,0.567635
5,superb,0.567303
6,deft,0.536631
7,spectacular,0.529938
8,streak,0.524905
9,scintillating,0.523787


The model associates "straight" with positive but generic traits (e.g., "brilliant", "impressive"), while "gay" is tied to specific stereotypes ("flamboyant"). This reflects a double standard where LGBTQ+ identities are narrowly defined.

In [50]:
analogy("rich", "mansion", "poor")

rich : mansion :: poor : ?


,Analogy word,Score
0,residence,0.596916
1,apartment,0.560366
2,buildings,0.552244
3,houses,0.548492
4,homes,0.547413
5,manor,0.541510
6,apartments,0.535705
7,bedroom,0.533607
8,house,0.528832
9,riverside,0.528188


In [52]:
analogy("republican", "conservative", "democrat")

republican : conservative :: democrat : ?


,Analogy word,Score
0,liberal,0.880956
1,centrist,0.737442
2,conservatives,0.726892
3,liberals,0.679743
4,politician,0.677826
5,socialist,0.664741
6,democratic,0.647387
7,tory,0.646461
8,populist,0.638959
9,incumbent,0.632339


The model strongly associates "democrat" with "liberal", which aligns with common U.S. political discourse. But, "Socialist" reflects right-wing framing of Democrats (common in conservative media), not necessarily the party’s range instance.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.5 Discussion
rubric={points}

**Your tasks:**
1. Discuss your observations from 1.4. Are there any worrisome biases in these embeddings trained on Wikipedia?   
2. Give an example of how using embeddings with biases could cause harm in the real world.

<div class="alert alert-warning">

Solution_1_5
    
</div>

_Points:_ 4

Biases in embeddings reflect and amplify societal inequalities. Proactive mitigation is really important to prevent harm in AI applications. in the examples used in 1.4, the exploration of biases in the Wikipedia-trained GloVe embeddings revealed several concerning patterns that reflect and amplify real-world stereotypes. The model consistently associated women and LGBTQ+ individuals with appearance or emotional traits (e.g., "she : beauty" scoring 0.609 in the genius analogy, "gay : flamboyant" linking to stereotypes), while men and heterosexual identities were tied to intellectual or leadership qualities. Racial biases appeared strongly, with Black identities showing higher similarity to crime-related terms (0.436) compared to white identities (0.38), perpetuating harmful media-driven narratives. Biases like these, could have an effect in situations like resume-ranking systems where it deprioritize female candidates for engineering jobs or male candidates for nursing roles, reinforcing industry gender gaps. Similarly, overassociating racial identities with crime compared to others could skew risk-assessment tools used in the police. 

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The goal of topic modeling is discovering high-level themes in a large collection of texts. 

In this homework, you will explore topics in [the 20 newsgroups text dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [53]:
from sklearn.datasets import fetch_20newsgroups

In [54]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [55]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)
rubric={points}

Preprocessing is a crucial step before carrying out topic modeling and it markedly affects topic modeling results. In this exercise, you'll prepare the data using [spaCy](https://spacy.io/) for topic modeling. 

**Your tasks:** 

- Write code using [spaCy](https://spacy.io/) to preprocess the `text` column in the given dataframe `df` and save the processed text in a new column called `text_pp` within the same dataframe.

If you do not have spaCy in your course environment, you'll have to [install it](https://spacy.io/usage) and download the pretrained model en_core_web_md. 

`python -m spacy download en_core_web_md`


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are likely to be more informative for the given task. Some common text preprocessing steps for topic modeling include: 
- getting rid of slashes, new-line characters, or any other non-informative characters
- sentence segmentation and tokenization      
- replacing urls, email addresses, or numbers with generic tokens such as "URL",  "EMAIL", "NUM". 
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization


> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://universaldependencies.org/u/pos/) is the list of part-of-speech tags used in spaCy. 

> You may have to use regex to clean text before passing it to spaCy. Also, you might have to go back and forth between preprocessing in this exercise and and topic modeling in Exercise 2 before finalizing preprocessing steps. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you finalize the preprocessing part, you might want to save the preprocessed data in a CSV and work with this CSV so that you don't run the preprocessing part every time you run the notebook. 
 


In [59]:
import spacy
nlp = spacy.load("en_core_web_md", disable=["parser", "ner"])

<div class="alert alert-warning">

Solution_2_1
    
</div>

_Points:_ 8

In [65]:
from tqdm import tqdm
import spacy
import re 
def preprocess_text(text):
    """Preprocess a single text document"""
    text = re.sub(r'http\S+|www\S+|https\S+', 'URL', text, flags=re.MULTILINE)
    text = re.sub(r'\S+@\S+', 'EMAIL', text)
    text = re.sub(r'\b\d+\b', 'NUM', text)
    doc = nlp(text)
    tokens = []
    for token in doc:
        if (token.is_stop or token.is_punct or token.is_space or 
            len(token.text) <= 2):
            continue
        lemma = token.lemma_.lower().strip()
        if lemma:
            tokens.append(lemma)
    
    return " ".join(tokens)

tqdm.pandas()
df['text_pp'] = df['text'].progress_apply(preprocess_text)
df.to_csv('preprocessed_newsgroups.csv', index=False)

100%|███████████████████████████████████████| 4563/4563 [01:20<00:00, 56.51it/s]


In [66]:
...

Ellipsis

In [67]:
...

Ellipsis

In [68]:
df.iloc[2:6]

,text,target,target_name,text_pp
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...
5,"Hi,\n I'd like to subscribe to Leadership Ma...",5,soc.religion.christian,like subscribe leadership magazine wonder disk...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification
rubric={points}

**Your tasks:**

- Outline the preprocessing steps you carried out in the previous exercise (bullet point format is fine), providing a brief justification when necessary. 

> You might want to wait to answer this question till you are done with Exercise 2 and you have finalized the preprocessing steps in 2.1. 

<div class="alert alert-warning">

Solution_2_2
    
</div>

_Points:_ 2

For URL/Email/Number Replacement: I replaced with URL, EMAIL, NUM tokens so that since they are low information tokens about the text, and they could distort the topics. 

spaCy Tokenization and Lemmatization: I split the text into tokens and reduce them into their base forms ("running" to "run"), so that the variants of the same word are normalized. 

Stopword Removal: I removed common words that are not that meaningful for understanding the topic and create noise  ("the", "and").

Removed all punctuation and extra spaces since they don’t contribute to topic meaning.

Short Token Filtering by excluding tokens with less or equal than 2 characters since most of them are not informative like 'it'.

finally lowercasing all the text so that the same words with different cases are treated identically.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation
rubric={points}

**Your tasks:**

1. Build LDA models on the preprocessed data using using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html) and random state 42. Experiment with a few values for the number of topics (`n_components`). Pick a reasonable number for the number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

_Points:_ 4

_Type your answer here, replacing this text._

In [71]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000)
X_counts = vectorizer.fit_transform(df['text_pp'])

n_topics_list = [5, 8, 10]  
lda_models = {}

for n_topics in n_topics_list:
    lda = LatentDirichletAllocation(
        n_components=n_topics,
        random_state=42,
        learning_method='online',
        n_jobs=-1
    )
    lda.fit(X_counts)
    lda_models[n_topics] = lda
    print(f"Trained LDA with {n_topics} topics")

best_n_topics = 8
lda = lda_models[best_n_topics]

Trained LDA with 5 topics
Trained LDA with 8 topics
Trained LDA with 10 topics


In [72]:
def print_top_words(model, feature_names, n_top_words=10):
    for topic_idx, topic in enumerate(model.components_):
        message = f"Topic #{topic_idx}: "
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words-1:-1]])
        print(message)

print("Top words per topic:")
print_top_words(lda, vectorizer.get_feature_names_out())

Top words per topic:
Topic #0: people gun armenian kill israel armenians right government turkish say
Topic #1: num period pts van power new cal det bos pit
Topic #2: god people believe jesus think question know say church religion
Topic #3: book article system university number new research public issue problem
Topic #4: game team year play player win season good hockey num
Topic #5: image file email window use program available server num widget
Topic #6: know think like go come time say get thing look
Topic #7: email file entry send program output section mail address info


I choosed 8 newsgroup categories since 8 allows direct comparison between discovered topics and known labels.
Models with 5 categories produced overly broad topics and models with 10 categories, creating redundant splits, like splitting a same topic. 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association
rubric={points}

**Your tasks:**
1. For the number of topics you picked in the previous exercise, show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

_Points:_ 5

topic 0: Middle East & Armenian Conflict

topic 1: Sports Statistics

topic 2: Religious Debate

topic 3: Academic Publishing

topic 4: Ice Hockey

topic 5: Computer Systems

topic 6: Discussion and debate

topic 7: Email/Software 

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association
rubric={points}

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

_Points:_ 5

In [74]:

topic_distributions = lda.transform(X_counts[:5])
dominant_topics = topic_distributions.argmax(axis=1)

for i, (text, topic_prob) in enumerate(zip(df['text'].head(), topic_distributions)):
    print(f"\nDocument {i} (Actual: {df['target_name'].iloc[i]}):")
    print(f"Dominant Topic: {dominant_topics[i]}")
    print("Topic Probabilities:", [f"{p:.3f}" for p in topic_prob])


Document 0 (Actual: talk.politics.guns):
Dominant Topic: 0
Topic Probabilities: ['0.516', '0.131', '0.003', '0.003', '0.003', '0.003', '0.339', '0.003']

Document 1 (Actual: comp.graphics):
Dominant Topic: 4
Topic Probabilities: ['0.002', '0.066', '0.002', '0.257', '0.405', '0.262', '0.002', '0.002']

Document 2 (Actual: comp.graphics):
Dominant Topic: 6
Topic Probabilities: ['0.003', '0.003', '0.003', '0.003', '0.003', '0.312', '0.671', '0.003']

Document 3 (Actual: comp.windows.x):
Dominant Topic: 5
Topic Probabilities: ['0.008', '0.008', '0.324', '0.008', '0.008', '0.626', '0.008', '0.008']

Document 4 (Actual: talk.politics.mideast):
Dominant Topic: 0
Topic Probabilities: ['0.958', '0.006', '0.006', '0.006', '0.006', '0.006', '0.006', '0.006']


1. Document 0 (Actual: talk.politics.guns)
Interpretation:
The model is uncertain between Topic 0 (likely gun politics) and Topic 6 (generic discussion).
Suggests Topic 0 might be too broad.

2. Document 1 (Actual: comp.graphics)
Interpretation:
Split assignment across tech-related topics (4, 5, 3).
Suggests overlapping vocabulary between graphics, windows, and academic topics.

3. Document 2 (Actual: comp.graphics)
Interpretation:
Misassigned to Topic 6 (generic discussion) despite being a tech document.
Likely due to too much noise text and few text.

4. Document 3 (Actual: comp.windows.x)
Interpretation:
Correctly assigned to a tech topic (5) but confused with academic terms (Topic 3).
Suggests Topics 3 and 5 need better separation.

5. Document 4 (Actual: talk.politics.mideast)
Interpretation:
Strong, clear assignment to Topic 0 (Middle East/gun politics blend).
Suggests the model can be confident when topics are distinct.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

rubric={points}

1. Briefly explain how content-based filtering works in the context of recommender systems. 
2. Discuss at least two negative consequences of recommender systems.
3. What is transfer learning in natural language processing? Briefly explain.     

<div class="alert alert-warning">

Solution_3
    
</div>

_Points:_ 6

1. Content-Based Filtering in Recommender Systems:

It recommends items by analyzing their features and matching them to a user’s preferences based on past behavior. For example, if a user frequently watches romance movies, the system will recommend other movies tagged with "romance" or containing similar plot features. Collaborative filtering relies on user-item interactions, using item attributes (genre, text descriptions) and user profiles, making it effective for new users or items but limits by the quality of feature representation.

2. Negative Consequences of Recommender Systems:

Filter bubbles: Systems reinforce existing preferences by recommending similar content, limiting exposure to other perspectives and views. 

Bias amplification: Historical biases in training data can lead to discriminatory recommendations. Recommender systems may prioritize addictive or extreme content to maximize engagement.

3. Transfer Learning in NLP:
   
Transfer learning makes pre-trained language models better, by training on large corpora to solve downstream tasks with minimal task-specific data. Instead of training from scratch, the model fine-tunes its pre-learned representations on a smaller labeled dataset for tasks like sentiment analysis or entity recognition. Reducing computational costs and improving performance. 



<!-- END QUESTION -->

<br><br><br><br>

**Before submitting your assignment, please make sure you have followed all the instructions in the Submission instructions section at the top.** 

![](img/eva-well-done.png)